In [36]:
import numpy as np
import pandas as pd
import re
import os
import sys

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

In [23]:
with open("sent_corpus.csv", "r") as sent_file:
    lines = sent_file.read().split("\n")

rows = [line.split(",") for line in lines if line]
rows = [row[:3] + [",".join(row[3:])] for row in rows]
# remove document start character
rows[0][0] = rows[0][0][1:]
sentDf_cols = ['ItemID', 'Sentiment', 'SentimentSource', 'SentimentText']
sentDf = pd.DataFrame(rows[1:],columns=sentDf_cols)
print(sentDf.columns.values)

['ItemID' 'Sentiment' 'SentimentSource' 'SentimentText']


In [27]:

sentDf['SentimentText'] = sentDf['SentimentText'].apply(lambda x: x.lower())
sentDf['SentimentText'] = sentDf['SentimentText'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(sentDf[ sentDf['Sentiment'] == '1'].size)
print(sentDf[ sentDf['Sentiment'] == '0'].size)
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(sentDf['SentimentText'].values)
X = tokenizer.texts_to_sequences(sentDf['SentimentText'].values)
X = pad_sequences(X)

3160740
3153768


In [105]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(Dropout(0.2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 40, 128)           256000    
_________________________________________________________________
dropout_7 (Dropout)          (None, 40, 128)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [47]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1057680, 40) (1057680,)
(520947, 40) (520947,)


Training the model takes about 1 hour.

In [48]:
batch_size = 32
model.fit(X_train, Y_train, initial_epoch=7, batch_size=batch_size, verbose=2)

Epoch 8/10
 - 1353s - loss: 0.4480 - acc: 0.7886
Epoch 9/10
 - 1349s - loss: 0.4226 - acc: 0.8038
Epoch 10/10
 - 1345s - loss: 0.4134 - acc: 0.8093


Testing on the test set takes about 3 minutes.

In [107]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
loss,acc = model.evaluate(X_test, Y_test, verbose = 1, batch_size = batch_size)

514947/514947 [==============================] - 183s 354us/step


In [108]:
print(model.metrics_names)
print("loss: %.2f" % (loss))
print("acc: %.2f" % (acc))

['loss', 'acc']
loss: 0.69
acc: 0.51


In [109]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
    if np.argmax(result) == Y_validate.values[x]:
        if Y_validate.values[x] == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if Y_validate.values[x] == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 72.17741935483872 %
neg_acc 27.91005291005291 %
